Plot Antarctic Cotidal Charts
=============================

This ({nb-download}`notebook <Plot-Antarctic-Cotidal-Charts.ipynb>`) demonstrates plotting cotidal charts for selected constituents around Antarctica

OTIS format tidal solutions provided by Oregon State University and ESR  
- [http://volkov.oce.orst.edu/tides/region.html](http://volkov.oce.orst.edu/tides/region.html) 
- [https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/](https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/)
- [ftp://ftp.esr.org/pub/datasets/tmd/](ftp://ftp.esr.org/pub/datasets/tmd/)  

Global Tide Model (GOT) solutions provided by Richard Ray at GSFC  
- [https://earth.gsfc.nasa.gov/geo/data/ocean-tide-models](https://earth.gsfc.nasa.gov/geo/data/ocean-tide-models)

Finite Element Solution (FES) provided by AVISO  
- [https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html](https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html)

## Python Dependencies
 - [numpy: Scientific Computing Tools For Python](https://www.numpy.org)  
 - [scipy: Scientific Tools for Python](https://www.scipy.org/)  
 - [pyproj: Python interface to PROJ library](https://pypi.org/project/pyproj/)  
 - [netCDF4: Python interface to the netCDF C library](https://unidata.github.io/netcdf4-python/)  
 - [matplotlib: Python 2D plotting library](http://matplotlib.org/)  
 - [cartopy: Python package designed for geospatial data processing](https://scitools.org.uk/cartopy/docs/latest/)  

## Program Dependencies

- `arguments.py`: load the nodal corrections for tidal constituents  
- `astro.py`: computes the basic astronomical mean longitudes  
- `crs.py`: Coordinate Reference System (CRS) routines  
- `io.model.py`: retrieves tide model parameters for named tide models
- `io.OTIS.py`: extract tidal harmonic constants from OTIS tide models  
- `io.ATLAS.py`: extract tidal harmonic constants from ATLAS netcdf models  
- `io.GOT.py`: extract tidal harmonic constants from GOT tide models  
- `io.FES.py`: extract tidal harmonic constants from FES tide models  
- `predict.py`: predict tidal values using harmonic constants  
- `time.py`: utilities for calculating time operations
- `utilities.py`: download and management utilities for files

This notebook uses Jupyter widgets to set parameters for calculating the cotidal maps.  

## Load modules

In [1]:
import pyproj
import numpy as np
import matplotlib
matplotlib.rcParams['axes.linewidth'] = 2.0
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# import tide programs
import pyTMD.io
import pyTMD.predict
import pyTMD.tools
import pyTMD.utilities

# autoreload
%load_ext autoreload
%autoreload 2

## Set parameters for program

- Model directory  
- Tide model  

In [ ]:
# available model list
model_list = sorted(pyTMD.io.model.ocean_elevation())
# display widgets for setting directory and model
TMDwidgets = pyTMD.tools.widgets()
TMDwidgets.model.options = model_list
TMDwidgets.model.value = 'CATS2008'
TMDwidgets.VBox([
    TMDwidgets.directory,
    TMDwidgets.model,
    TMDwidgets.compress,
])

## Setup tide model parameters

In [3]:
# get model parameters
model = pyTMD.io.model(TMDwidgets.directory.value,
    compressed=TMDwidgets.compress.value
   ).elevation(TMDwidgets.model.value)

## Setup coordinates for calculating tidal amplitudes

In [4]:
# create an image around Antarctica
xlimits = [-560.*5e3,560.*5e3]
ylimits = [-560.*5e3,560.*5e3]
spacing = [20e3,-20e3]
# x and y coordinates
x = np.arange(xlimits[0],xlimits[1]+spacing[0],spacing[0])
y = np.arange(ylimits[1],ylimits[0]+spacing[1],spacing[1])
xgrid,ygrid = np.meshgrid(x,y)
# x and y dimensions
nx = int((xlimits[1]-xlimits[0])/spacing[0])+1
ny = int((ylimits[0]-ylimits[1])/spacing[1])+1
# convert image coordinates from polar stereographic to latitude/longitude
crs1 = pyproj.CRS.from_epsg(3031)
crs2 = pyproj.CRS.from_epsg(4326)
transformer = pyproj.Transformer.from_crs(crs1, crs2, always_xy=True)
lon,lat = transformer.transform(xgrid.flatten(), ygrid.flatten())

## Calculate tide map

In [5]:
# read tidal constants and interpolate to grid points
if model.format in ('OTIS','ATLAS-compact','TMD3'):
    amp,ph,D,c = pyTMD.io.OTIS.extract_constants(lon, lat, model.grid_file,
        model.model_file, model.projection, type=model.type, crop=True,
        method='spline', grid=model.file_format)
elif (model.format == 'ATLAS-netcdf'):
    amp,ph,D,c = pyTMD.io.ATLAS.extract_constants(lon, lat, model.grid_file,
        model.model_file, type=model.type, crop=True, method='spline',
        scale=model.scale, compressed=model.compressed)
elif model.format in ('GOT-ascii', 'GOT-netcdf'):
    amp,ph,c = pyTMD.io.GOT.extract_constants(lon, lat, model.model_file,
        grid=model.file_format, crop=True, method='spline', scale=model.scale,
        compressed=model.compressed)
elif (model.format == 'FES-netcdf'):
    amp,ph = pyTMD.io.FES.extract_constants(lon, lat, model.model_file,
        type=model.type, version=model.version, crop=True,
        method='spline', scale=model.scale, compressed=model.compressed)
    c = model.constituents

## Select constituent for cotidal chart

In [ ]:
TMDwidgets.constituents.options = c
TMDwidgets.constituents.value = c[0]
display(TMDwidgets.constituents)

## Create cotidal chart

In [ ]:
# extract amplitude and phase for constituent
i = TMDwidgets.constituents.index
cons = TMDwidgets.constituents.value
amplitude = amp[:,i].reshape(ny,nx)
phase = ph[:,i].reshape(ny,nx)

# plot Antarctic cotidal charts
projection = ccrs.Stereographic(central_longitude=0.0,
    central_latitude=-90,true_scale_latitude=-71.0)
fig, ax = plt.subplots(num=1, figsize=(9,8),
    subplot_kw=dict(projection=projection))
# plot tide amplitude
extent = (xlimits[0],xlimits[1],ylimits[0],ylimits[1])
im = ax.imshow(amplitude, interpolation='nearest',
    vmin=0, vmax=amplitude.max(), transform=projection,
    extent=extent, origin='upper')
# plot tide phase
contour_levels = np.arange(0, 360, 30)
contour = ax.contour(xgrid, ygrid, phase,
    contour_levels, colors='0.2', linestyles='solid',
    transform=projection)
# add high resolution cartopy coastlines
ax.coastlines('10m')

# Add colorbar and adjust size
# pad = distance from main plot axis
# extend = add extension triangles to upper and lower bounds
# options: neither, both, min, max
# shrink = percent size of colorbar
# aspect = lengthXwidth aspect of colorbar
cbar = plt.colorbar(im, ax=ax, pad=0.025, extend='max',
    extendfrac=0.0375, shrink=0.85, aspect=22.5, drawedges=False)
# rasterized colorbar to remove lines
cbar.solids.set_rasterized(True)
# Add label to the colorbar
cbar.ax.set_ylabel(f'{model.name} {cons} Tide Chart', labelpad=10, fontsize=13)
cbar.ax.set_title('cm', fontsize=13, va='bottom')
# ticks lines all the way across
cbar.ax.tick_params(which='both', width=1, length=20,
    labelsize=13, direction='in')

# set x and y limits
ax.set_xlim(xlimits)
ax.set_ylim(ylimits)

# stronger linewidth on frame
ax.spines['geo'].set_linewidth(2.0)
ax.spines['geo'].set_capstyle('projecting')
# adjust subplot within figure
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)

# show the plot
plt.show()